### Install, import and read files.

In [ ]:
%pip install pandas

In [1]:
import numpy as np
import pandas as pd 
# import s3fs

what_corpus = 'data_total'  # 'polnear', 'parc30'
what_type_files = 'test'

# filepath = 's3://sagemaker-studio-528576943967-ssf9zkrg3os/polnear-conll/prepared/'
filepath = '../' + what_corpus + '-conll/prepared/'

In [2]:
filename = what_corpus + '_preprocessed_' + what_type_files + '_noBIO.csv'
dataset = pd.read_csv(filepath + filename, sep=";")

In [3]:
# Loop over testfiles, loop over tokens, first look for CUE sequences and number them. Then find the closest SOURCE and CONTENTS sequences and give them the same number.
# If we loop over the file in dict format, we can change the dict easily during the loop.
# For finding SOURCE and CONTENT do an additional loop traveling the dict backwards and forwards at the same time with loop_index and -loop_inde

In [4]:
class FilesGetter(object):
    
    def __init__(self, dataset):
        self.n_files = 1
        self.dataset = dataset
        self.empty = False
#         agg_func = lambda f: ' '.join(f["word"].values.tolist())
        agg_func = lambda f: [(w, f) for w, f in zip(f["word"].values.tolist(),
                                                     f["tag"].values.tolist())]
        self.grouped = self.dataset.groupby(["filename"]).apply(agg_func)
        self.files = [f for f in self.grouped]
    
    def get_next(self):
        try:
            f = self.grouped["Files: {}".format(self.n_files)]
            self.n_files += 1
            return f
        except:
            return None

In [5]:
files_getter = FilesGetter(dataset)

In [6]:
files = files_getter.files

In [7]:
print(len(files))
# files[0:1]
# # [[('Washington', 'SOURCE'),
# #   ('Post', 'SOURCE'),
# #   ('back', 'CUE'),
# #   ('Hillary', 'CONTENT'),
# #   ('Clinton', 'CONTENT'),
# #   ('for', 'CONTENT'),
# #   ('president', 'CONTENT'),
# #   ('.', 'O'),
# #   ('Washington', 'O'),
# #   ('-lrb-', 'O'),
# #   ('AFP', 'O'),
# #   ('-rrb-', 'O'),
# #   ('--', 'O'),
# #   ('the', 'O'),
# #   ('Washington', 'O'),

184


In [32]:
# testlist = [1, 2, 3, 4, 5, 6]
# start_index = 2
# print(testlist[start_index])
# testlist.reverse()
# start_index = len(testlist) - start_index - 1
# print(start_index)
# print(testlist[start_index])

# for file_index, file in enumerate(file_temp[start_index:]):
#     print(file_index)

3
3
3
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26


In [65]:
start_index_labels = 0  # = 2  # For BIO labels

def number_continuous_labels(file_temp, start_index, label_nr, label_type, reverse=False):
    if reverse:
        file_temp.reverse()
        start_index = len(file_temp) - start_index  # - 1  # - 1 to correct for list starting at 0
        
    for file_index, file in enumerate(file_temp):
        if file_index > start_index:
#             print(file_index)
            label = file[1]
            if label == label_type:
                token = file[0]
                file_temp[file_index] = (token, label + '-' + str(label_nr))
            else:
                break

    # Reverse back once we're done
    if reverse:
        file_temp.reverse()
        
    return file_temp

# Find the CONTENT or SOURCE label for a CUE and add the number to the label
def number_label(token_idx, file_temp, label_nr, label_type):
    label_search_index = 1
    label_found_index = None
    
#     print(label_type)
#     print(file_temp)
    
    file_temp_len = len(file_temp)
    backwards_search_index = token_idx - label_search_index
    forwards_search_index = token_idx + label_search_index

#     print(file_temp_len)
#     print(backwards_search_index)
#     print(forwards_search_index)
#     print(len(file_temp[0:backwards_search_index]) > 0 or len(file_temp[forwards_search_index:file_temp_len]) > 0)

    while len(file_temp[0:backwards_search_index]) > 0 or len(file_temp[forwards_search_index:file_temp_len]) > 0:
#         print(label_search_index)
        backwards_search_index = token_idx - label_search_index
        forwards_search_index = token_idx + label_search_index

        # Backwards loop
        if backwards_search_index >= 0:
            file_temp_item_backwards = file_temp[backwards_search_index]
#             print('backwards')
#             print(file_temp_item_backwards)
            label_backwards = file_temp_item_backwards[1]  # Label is 2nd in tuple
            if label_backwards[start_index_labels:] == label_type:
#                 print('label found')
                # Overwrite a possible content_found_index in the forward loop. 
                # We favor the backwards content_found_index over the forward loop.
                # If we find a content, break immediately from while loop
                label_found_index = backwards_search_index
                token_backwards  = file_temp_item_backwards[0]  # Token is 1st in tuple
                file_temp[label_found_index] = (token_backwards, label_backwards + '-' + str(label_nr))
#                 print(file_temp[label_found_index])
                file_temp = number_continuous_labels(file_temp, label_found_index, label_nr, label_type, reverse=True)
                break

        # Forwards loop
        if forwards_search_index < file_temp_len:
            file_temp_item_forwards = file_temp[forwards_search_index]
#             print('forwards')
#             print(file_temp_item_forwards)
            label_forwards = file_temp_item_forwards[1]  # Label is 2nd in tuple
            if label_forwards[start_index_labels:] == label_type: 
#                 print('label found')
                label_found_index = forwards_search_index
                token_forwards  = file_temp_item_forwards[0]  # Token is 1st in tuple
                file_temp[label_found_index] = (token_forwards, label_forwards + '-' + str(label_nr))
#                 print(file_temp[label_found_index])
                file_temp = number_continuous_labels(file_temp, label_found_index, label_nr, label_type)
                break

        label_search_index += 1 

        if label_search_index > file_temp_len:
            break

        # Uncomment for debugging
        if label_search_index == 1:  # One is first sentence with labels
            break 

    return file_temp

In [69]:
files_labels_numbered = []
for file_index, file in enumerate(files):
#     file = file[0:30]  # Uncomment for debugging
    # Copy of the tokens in the file
    file_temp = file.copy()
#     file_temp_len = len(file_temp)
    label_nr = 1
#     print('file')
#     print(file)
#     print(file_temp)
#     print(file_temp_len)
#     print(label_nr)
    
    # Loop over tokens to number labels
    cue_found_index = -1
    for token_idx, file_item in enumerate(file):
        label = file_item[1]  # Label is 2nd in tuple
        
#         print('token')
#         print(token_idx)
#         print(label)
        
        # If the label is CUE without a number, number the CUE first
        is_search_content_source = False
        if label[start_index_labels:] == 'CUE':
#             print('cue found')
#             print(cue_found_index)
#             print(token_idx > cue_found_index+1)

            # We are only handling continuous CUEs. If CUE is not continuous, then the label nr goes up 
            if token_idx > cue_found_index + 1:
                label_nr += 1
                is_search_content_source = True
                
            token = file_item[0]  # Token is 1st in tuple
            file_temp[token_idx] = (token, label + '-' + str(label_nr))
            cue_found_index = token_idx
#             print(file_temp[token_idx])
            
            if is_search_content_source:
#                 print('label_nr')
#                 print(label_nr)
                
#                 if label_nr == 3:
                # Now search backwards and forwards in file for CONTENT and SOURCE labels
                file_temp = number_label(token_idx, file_temp, label_nr, 'CONTENT')
                file_temp = number_label(token_idx, file_temp, label_nr, 'SOURCE')
                
            files_labels_numbered.append(file_temp)
            
#     if file_index == 0:
#         break
                
files_labels_numbered[1]

[('Washington', 'SOURCE'),
 ('Post', 'SOURCE-2'),
 ('back', 'CUE-2'),
 ('Hillary', 'CONTENT-2'),
 ('Clinton', 'CONTENT-2'),
 ('for', 'CONTENT-2'),
 ('president', 'CONTENT-2'),
 ('.', 'O'),
 ('Washington', 'O'),
 ('-lrb-', 'O'),
 ('AFP', 'O'),
 ('-rrb-', 'O'),
 ('--', 'O'),
 ('the', 'O'),
 ('Washington', 'O'),
 ('Post', 'O'),
 ('on', 'O'),
 ('Thursday', 'O'),
 ('become', 'O'),
 ('the', 'SOURCE-3'),
 ('latest', 'SOURCE-3'),
 ('US', 'SOURCE-4'),
 ('newspaper', 'SOURCE-3'),
 ('to', 'CUE-3'),
 ('emphatically', 'CUE-3'),
 ('endorse', 'CUE-3'),
 ('Hillary', 'CONTENT-3'),
 ('Clinton', 'CONTENT-3'),
 ('for', 'CONTENT-3'),
 ('the', 'CONTENT-3'),
 ('White', 'CONTENT-3'),
 ('House', 'CONTENT-3'),
 (',', 'O'),
 ('say', 'CUE-4'),
 ('it', 'CONTENT-4'),
 ('be', 'CONTENT-4'),
 ('sway', 'CONTENT-4'),
 ('as', 'CONTENT-4'),
 ('much', 'CONTENT-4'),
 ('by', 'CONTENT-4'),
 ('she', 'CONTENT-4'),
 ('competence', 'CONTENT-4'),
 ('as', 'CONTENT-4'),
 ('by', 'CONTENT-4'),
 ('the', 'CONTENT-4'),
 ('alarming', 'CON

In [70]:
files_labels_numbered[2]

[('Washington', 'SOURCE'),
 ('Post', 'SOURCE-2'),
 ('back', 'CUE-2'),
 ('Hillary', 'CONTENT-2'),
 ('Clinton', 'CONTENT-2'),
 ('for', 'CONTENT-2'),
 ('president', 'CONTENT-2'),
 ('.', 'O'),
 ('Washington', 'O'),
 ('-lrb-', 'O'),
 ('AFP', 'O'),
 ('-rrb-', 'O'),
 ('--', 'O'),
 ('the', 'O'),
 ('Washington', 'O'),
 ('Post', 'O'),
 ('on', 'O'),
 ('Thursday', 'O'),
 ('become', 'O'),
 ('the', 'SOURCE-3'),
 ('latest', 'SOURCE-3'),
 ('US', 'SOURCE-4'),
 ('newspaper', 'SOURCE-3'),
 ('to', 'CUE-3'),
 ('emphatically', 'CUE-3'),
 ('endorse', 'CUE-3'),
 ('Hillary', 'CONTENT-3'),
 ('Clinton', 'CONTENT-3'),
 ('for', 'CONTENT-3'),
 ('the', 'CONTENT-3'),
 ('White', 'CONTENT-3'),
 ('House', 'CONTENT-3'),
 (',', 'O'),
 ('say', 'CUE-4'),
 ('it', 'CONTENT-4'),
 ('be', 'CONTENT-4'),
 ('sway', 'CONTENT-4'),
 ('as', 'CONTENT-4'),
 ('much', 'CONTENT-4'),
 ('by', 'CONTENT-4'),
 ('she', 'CONTENT-4'),
 ('competence', 'CONTENT-4'),
 ('as', 'CONTENT-4'),
 ('by', 'CONTENT-4'),
 ('the', 'CONTENT-4'),
 ('alarming', 'CON